In [1]:
import numpy as np
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import initializers, losses

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # carregar dados


In [4]:
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
image_size = x_train.shape[1] # salvando o tamanho da imagem
x_train = x_train.astype('float32') / 255 # normalizacao
x_test = x_test.astype('float32') / 255 # normalizacao

(50000, 32, 32, 3)


In [5]:
[n,height,width,numChannels] = x_train.shape
input_shape = (height, width, numChannels) # formato da camada de entrada
batch_size = 32
kernel_size = 3 # kernel 3x3
pool_size = 2
filters = 32
dropout = 0.2 # dropout rate

In [6]:
model = Sequential()
# Foram utilizadas dois conjuntos de camadas compostas por uma camada convolucional com função de ativacao ReLu e uma camada de max pooling

# ========= primeiro conjunto =========================== 
model.add(Conv2D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu',
                 input_shape=input_shape))
# dim = (30x30x32)

model.add(MaxPooling2D(pool_size=pool_size))
# o pooling, como eh igual a 2, diminui a dimensao pela metade
# dim = (15x15x32)
# ========= primeiro conjunto ===========================

# ========= segundo conjunto ===========================
# neste conjunto foram utilizados o dobro de filtros
model.add(Conv2D(filters=2*filters,
                 kernel_size=kernel_size,
                 activation='relu'))
# dim = (13x13x64)

model.add(MaxPooling2D(pool_size=pool_size))
# dim = (6x6x64)
# ========= segundo conjunto ===========================

# foi entao adicionada mais uma camada de convolucao
model.add(Conv2D(filters=2*filters,
                 kernel_size=kernel_size,
                 activation='relu'))
# dim = (4x4x64)

# redefinindo a dimensao
model.add(Flatten())
# dim = (1024)

# reducao da dimensao com uma camada fully connected
model.add(Dense(units=2*filters,activation='softmax'))
# dim = (64)

# camada dropout para evitar o overfitting
model.add(Dropout(dropout))
# a camada dropout nao altera a dimensao

# camada de saida
model.add(Dense(units=num_labels,activation='sigmoid'))
# dim = (10)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [7]:
# foi utilizada a Categorical Crossentropy como loss function, visto queeh um problema com multiplas classes
# neste caso o otimizador Adam perfomou melhor do que o SGD, entao este foi escolhido
model.compile(loss=losses.CategoricalCrossentropy(from_logits=False),
              optimizer=Adam(),
              metrics=['accuracy'])

In [8]:
# a principio foram utilizadas 10 epocas para treinamento, visto que nao estava avendo uma variacao consideravel da acuracia com mais epocas
model.fit(x_train, y_train, epochs=10, batch_size=batch_size, validation_data=(x_test, y_test))

_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

Epoch 1/10
1563/1563 [==============================] - 28s 17ms/step - loss: 2.0883 - accuracy: 0.2168 - val_loss: 1.8802 - val_accuracy: 0.3304
Epoch 2/10
1563/1563 [==============================] - 29s 19ms/step - loss: 1.8352 - accuracy: 0.3149 - val_loss: 1.6716 - val_accuracy: 0.4031
Epoch 3/10
1563/1563 [==============================] - 29s 18ms/step - loss: 1.6526 - accuracy: 0.3966 - val_loss: 1.4757 - val_accuracy: 0.5102
Epoch 4/10
1563/1563 [==============================] - 29s 19ms/step - loss: 1.4788 - accuracy: 0.4750 - val_loss: 1.2737 - val_accuracy: 0.5823
Epoch 5/10
1563/1563 [==============================] - 30s 19ms/step - loss: 1.3610 - accuracy: 0.5088 - val_loss: 1.2242 - val_accuracy: 0.5916
Epoch 6/10
1563/1563 [==============================] - 33s 21ms/step - loss: 1.3000 - accuracy: 0.5273 - val_loss: 1.1491 - val_accuracy: 0.6116
Epoch 7/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.2456 - accuracy: 0.5489 - val_loss: 1.1207 -

In [17]:
# neste trecho foi feita uma tentativa de classificar uma imagem de cachorro usando a rede construida

import cv2
import tensorflow as tf

# leitura da imagem
image = cv2.imread("dog_32.jpg")

# mudanca do formato para atender as restricoes de entrada da rede
image = tf.reshape(image,[-1,32,32,3])
result = model.predict(image)
result = np.argmax(result)

# o resultado obtido foi a classe 5, justamente a que representa o cachorro
print(result)

1/1 [==============================] - 0s 21ms/step
5
